<a href="https://colab.research.google.com/github/deedee-ke/practice/blob/main/Chapter_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
from abc import ABC, abstractmethod
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')


class LineItem:
    def __init__(self, product, quantity, price):
        self.product = product
        self.quantity = quantity
        self.price = price

    def total(self):
        return self.price * self.quantity


class Order:  # the Context
    def __init__(self, customer, cart, promotion=None):
        self.customer = customer
        self.cart = list(cart)
        self.promotion = promotion

    def total(self):
        if not hasattr(self, '__total'):
            self.__total = sum(item.total() for item in self.cart)
        return self.__total

    def due(self):
        if self.promotion is None:
            discount = 0
        else:
            discount = self.promotion.discount(self)
        return self.total() - discount

    def __repr__(self):
        fmt = '<Order total: {:.2f} due: {:.2f}>'
        return fmt.format(self.total(), self.due())


class Promotion(ABC):  # the Strategy: an abstract base class
    @abstractmethod
    def discount(self, order):
        """Return discount as a positive dollar amount"""


class FidelityPromo(Promotion):  # first Concrete Strategy
    """5% discount for customers with 1000 or more fidelity points"""
    def discount(self, order):
        return order.total() * .05 if order.customer.fidelity >= 1000 else 0


class BulkItemPromo(Promotion):  # second Concrete Strategy
    """10% discount for each LineItem with 20 or more units"""
    def discount(self, order):
        discount = 0
        for item in order.cart:
            if item.quantity >= 20:
                discount += item.total() * .1
        return discount


class LargeOrderPromo(Promotion):  # third Concrete Strategy
    """7% discount for orders with 10 or more distinct items"""
    def discount(self, order):
        distinct_items = {item.product for item in order.cart}
        if len(distinct_items) >= 10:
            return order.total() * .07
        return 0


In [2]:
joe = Customer('John Doe', 0)
ann = Customer('Ann Smith', 1100)
cart = [LineItem('banana', 4, .5),
        LineItem('apple', 10, 1.5),
        LineItem('watermellon', 5, 5.0)]
Order(joe, cart, FidelityPromo())

<Order total: 42.00 due: 42.00>

In [3]:
Order(ann, cart, FidelityPromo())

<Order total: 42.00 due: 39.90>

In [4]:
banana_cart = [LineItem('banana', 30, .5),
               LineItem('apple', 10, 1.5)]

Order(joe, banana_cart, BulkItemPromo())

<Order total: 30.00 due: 28.50>

In [5]:
long_order = [LineItem(str(item_code), 1, 1.0)
             for item_code in range(10)]
Order(joe, long_order, LargeOrderPromo())

<Order total: 10.00 due: 9.30>

In [6]:
Order(joe, cart, LargeOrderPromo())

<Order total: 42.00 due: 42.00>

In [7]:
from collections import namedtuple

Customer = namedtuple('Customer', 'name fidelity')

class LineItem:

  def _init_(self, product, quantity, price):
    self.product = product
    self.quantity = quantity
    self.price = price

  def total(self):
    return self.price * self.quantity

class Order:

  def _init_(self, customer, cart, promotion=None):
    self.customer = customer
    self.cart = list(cart)
    self.promotion = promotion

  def total(self):
    if not hasattr(self, '__total'):
      self.__total = sum(item.total() for item in self.cart)
    return self.__total

  def due(self):
    if self.promotion is None:
      discount = 0
    else:
      discount = self.promotion(self)
    return self.total() - discount

  def _repr_(self):
    fmt = '<Order total: {:.2f} due: {:.2f}>'
    return fmt.format(self.total(), self.due())

def fidelity_promo(order):
  """5% discount for customers with 1000 or more fidelity points"""
  return order.total() * .05 if order.customer.fidelity >= 1000 else 0

def bulk_item_promo(order):
  """10% discount for each LineItem with 20 or more units"""
  discount = 0
  for item in order.cart:
    if item.quantity >= 20:
      discount += item.total() * .1

def large_order_promo(order):
  """7% discount for orders with 10 or more distinct items"""
  distinct_items = {item.product for item in order.cart}
  if len(distinct_items) >= 10:
    return order.total() * .07
  return 0


In [13]:
Order(joe, long_order, best_promo)

AttributeError: ignored

In [12]:
promos = [fidelity_promo, bulk_item_promo, large_order_promo]

def best_promo(order):
  """Select best discount available"""
  return max(promo(order) for promo in promos)

In [14]:
promos = [globals()[name] for name in globals()
          if name.endswith('_promo')
          and name != 'best_promo']

def best_promo(order):
  """Select best discount available"""
  return max(promo(order) for promo in promos)


In [16]:
promos = [func for name, func in inspect.getmembers(promotions, inspect.isfunction)]

def best_promo(order):
  """Select best discount available"""
  return max(promo(order) for promo in promos)

NameError: ignored

In [17]:
class MacroCommand:
  """A comand that executes a list of commands"""

  def _init_(self, commands):
    self.commands = list(commands)

  def _call_(self):
    for command in self.commands:
      command()